In [1]:
%load_ext rpy2.ipython

C:\Users\deeth\miniconda3\envs\QA_eyetracking_env\lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [2]:
import rpy2

In [3]:
%%R 
# rpy2.library("devtools")
# rpy2.install_github("tmalsburg/scanpath/scanpath", dependencies=TRUE)
# rpy2.library(scanpath)

library(devtools)
install_github("tmalsburg/scanpath/scanpath", dependencies=TRUE)
library(scanpath)

R[write to console]: Loading required package: usethis

R[write to console]: Using GitHub PAT from the git credential store.

R[write to console]: Skipping install of 'scanpath' from a github remote, the SHA1 (6984b604) has not changed since last install.
  Use `force = TRUE` to force installation



In [5]:
import numpy as np
import pandas as pd
import rpy2.robjects as ro


def scasim_py(
    df: pd.DataFrame,
    formula: str,
    center_x: int = 1280,
    center_y: int = 720,
    viewing_distance: int = 77,
    unit_size: float = 1 / 60,
    normalize: bool | str = False,
) -> np.ndarray:
    ro.pandas2ri.activate()
    scanpath = ro.packages.importr(name="scanpath")

    with (ro.default_converter + ro.pandas2ri.converter).context():
        r_dataframe = ro.conversion.get_conversion().py2rpy(df)

    return scanpath.scasim(
        data=r_dataframe,
        formula=ro.Formula(formula),
        center_x=center_x,
        center_y=center_y,
        viewing_distance=viewing_distance,
        unit_size=unit_size,
        normalize=normalize,
    )


def alignment_plot_py(
    sp1: pd.DataFrame,
    sp2: pd.DataFrame,
    diff_by: str = "synthesized",
    center_x: int = 1280,
    center_y: int = 720,
    viewing_distance: int = 77,
    unit_size: float = 1 / 60,
):
    ro.pandas2ri.activate()
    scanpath = ro.packages.importr(name="scanpath")
    ggplot = ro.packages.importr("ggplot2")

    with (ro.default_converter + ro.pandas2ri.converter).context():
        sp_r_df_1 = ro.conversion.get_conversion().py2rpy(sp1)
        sp_r_df_2 = ro.conversion.get_conversion().py2rpy(sp2)

    alignment = scanpath.rscasim(
        sp_r_df_1,
        sp_r_df_2,
        formula=ro.Formula(
            f"CURRENT_FIX_DURATION ~ CURRENT_FIX_X + CURRENT_FIX_Y | {diff_by}"
        ),
        center_x=center_x,
        center_y=center_y,
        viewing_distance=viewing_distance,
        unit_size=unit_size,
    )

    alignment_plot = scanpath.plot_alignment(
        sp_r_df_1,
        sp_r_df_2,
        ro.conversion.get_conversion().py2rpy(alignment),
        ro.Formula(f"CURRENT_FIX_DURATION ~ CURRENT_FIX_X + CURRENT_FIX_Y | {diff_by}"),
        10,
        10,
    )

    # save the plot
    ggplot.ggsave(
        ro.conversion.get_conversion().py2rpy(alignment_plot), "alignment_plot.png"
    )


TypeError: unsupported operand type(s) for |: 'type' and 'type'

In [35]:
differentiate_trial_by_col = None
multiple_scanpaths_df = None

distances = scasim_py(
    df=multiple_scanpaths_df,
    formula=f"CURRENT_FIX_DURATION ~ CURRENT_FIX_X + CURRENT_FIX_Y | {differentiate_trial_by_col}",
)
